# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901995


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:07<01:54,  3.70s/it]

Rendering models:   9%|▉         | 3/33 [00:07<01:20,  2.69s/it]

Rendering models:  12%|█▏        | 4/33 [00:08<00:59,  2.04s/it]

Rendering models:  15%|█▌        | 5/33 [00:08<00:44,  1.58s/it]

...

Rendering models:  91%|█████████ | 30/33 [00:15<00:00,  3.60it/s]

Rendering models:  94%|█████████▍| 31/33 [00:15<00:00,  3.26it/s]

Rendering models:  97%|█████████▋| 32/33 [00:16<00:00,  3.13it/s]

Rendering models: 100%|██████████| 33/33 [00:16<00:00,  3.69it/s]

cmk24                                 0.001377
equidad1                              0.001129
klmasters                             0.001221
not-logged-in-8c0c44faf754d344b18a    0.001284
not-logged-in-88f20bcb6fdeadfc1772    0.007899
not-logged-in-783570dae2c2970c42b4    0.051781
CTidwell3                             0.001432
not-logged-in-d254cc3304773762c2aa    1.547332
Jes8x                                 0.002008
Prothon                               0.005872
ravendrop                             0.001224
Huskynator                            0.002065
firejuggler                           0.003685
not-logged-in-5a8bb7be433678ade709    0.004873
OmGupta                               0.001674
Kimby12                               0.003905
pangeli5                              0.001622
gabrielweiss                          0.011354
ElisabethB                            0.001118
skysearcher1                          0.008048
Lavadude                              0.021470
shiuhlin     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())